# Estimate Uncertainty

In this notebook we will calculate the uncertainty of each prompt(questions) from the created educational tutor(chatbot). To do so, we will use the methods of lexical similarity and eigenvalue Laplacian for both gpt-3.5 and gpt-4 models.

We load all necessary libraries and packages from lm_polygraph:

In [1]:
from lm_polygraph.utils.model import BlackboxModel
from lm_polygraph.utils.manager import estimate_uncertainty
from lm_polygraph.estimators import LexicalSimilarity, EigValLaplacian

c:\Users\35796\newanacoda\envs\env_lm_polygraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\35796\newanacoda\envs\env_lm_polygraph\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\35796\newanacoda\envs\env_lm_polygraph\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\35796\newanacoda\envs\env_lm_polygraph\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch

We will use our secret openAI API key to implement the following:

In [2]:
filename = 'apikey.txt'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(filename)        

We create a function to read the "input_messages.txt" textfile that contains all the given prompts and questions the user gave to the educational tutor.

In [3]:
# Function to read lines from the input file
def read_input_messages(filename):
    try:
        with open(filename, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
        return []


Read the questions from the "input_messages" textfile

In [4]:
# Read input messages from file
input_messages = read_input_messages('input_messages.txt')


## We will define the model and estimator 

## Lexical Similarity

A function to calculate the uncertainty for each prompt and then store it

In [5]:
import csv

# Function to estimate uncertainty for each prompt and save the results
def estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename):
    with open(output_filename, 'w', newline='', encoding='utf-8') as output_file:
        csv_writer = csv.writer(output_file)
        csv_writer.writerow(["Input Prompt", "Output Answer", "Uncertainty", "Model"])
        for prompt in input_messages:
            prompt = prompt.strip()  # Remove leading/trailing whitespaces
            uncertainty = estimate_uncertainty(model, estimator, input_text=prompt)
            csv_writer.writerow([prompt, uncertainty.generation_text, uncertainty.uncertainty, model.model_path])


### 1st case:
- Model: gpt-3.5-turbo
- Estimaotr: Lexical Similarity

In [6]:
model = BlackboxModel(
    api_key,
    'gpt-3.5-turbo'
)

estimator = LexicalSimilarity()

In [7]:
#TEST
# Estimate uncertainty for each prompt and save the results
output_filename = 'TEST.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

We store the data from this case in a csv file named: "results_LS_3.csv"

In [9]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_LS_3.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)


Here we run again the code to check for new uncertainty values in case 1: 

In [7]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_LS_3_new.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)


### 2nd case:
- Model: gpt-4
- Estimaotr: Lexical Similarity

In [8]:
model = BlackboxModel(
    api_key,
    'gpt-4'
)

estimator = LexicalSimilarity()

We store the data from this case in a csv file named: "results_LS_4.csv"

In [17]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_LS_4.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

Here we run again the code to check for new uncertainty values in case 2: 

In [10]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_LS_4_new.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

## EignValue Laplacian

Because using EignValue Laplacian as an estimator is more time consuming than Lexical Similarity we will test the uncertainty in the first 20 prompts. We will do some adjustments in the following function: 

In [5]:
import csv
from itertools import islice

# Function to estimate uncertainty for each prompt and save the results
def estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename):
    with open(output_filename, 'w', newline='', encoding='utf-8') as output_file:
        csv_writer = csv.writer(output_file)
        csv_writer.writerow(["Input Prompt", "Output Answer", "Uncertainty", "Model"])

        # Use itertools.islice to only read the first 20 lines
        for prompt in islice(input_messages, 20):
            prompt = prompt.strip()  # Remove leading/trailing whitespaces
            uncertainty = estimate_uncertainty(model, estimator, input_text=prompt)
            csv_writer.writerow([prompt, uncertainty.generation_text, uncertainty.uncertainty, model.model_path])


### 3nd case:
- Model: gpt-3.5-turbo
- Estimaotr: EigValLaplacian

In [12]:
model = BlackboxModel(
    api_key,
    'gpt-3.5-turbo'
)

estimator = EigValLaplacian(verbose=True)

We store the data from this case in a csv file named: "results_EVL_3.csv"

In [17]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_EVL_3.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

generated answers: ['Cyprus is a country located in the eastern Mediterranean region that plays a significant role in both international and national arenas. In terms of international relations, Cyprus is a member of several important organizations and alliances, including the United Nations, the European Union, the Commonwealth of Nations, and the Council of Europe. The country also has close ties with neighboring countries in the region, such as Greece, Turkey, and Israel.\n\nOne of the key issues that Cyprus faces on the international stage is the longstanding dispute over the', 'Cyprus is a small island nation located in the Eastern Mediterranean, with a unique geopolitical position that has significant implications in both the international and national arenas.\n\nIn the international arena, Cyprus has been a member of the United Nations since its independence in 1960. It is also a member of the European Union, which it joined in 2004. Additionally, Cyprus is a member of several i

generated answers: ['The European Union did not exist in 1964, so it did not have a formal reaction to the division of Cyprus along ethnic lines due to the EOKA terror campaign. However, individual European countries did express concern over the situation and called for a peaceful resolution to the conflict. The UN also played a role in trying to mediate the dispute between the Greek and Turkish Cypriots. Ultimately, a UN peacekeeping force was deployed to the island to maintain stability and prevent further violence.', 'When Cyprus was divided along ethnic lines in 1964 due to the EOKA terror campaign, the European Union did not yet exist. The EU was officially established in 1993, nearly three decades after the events in Cyprus.\n\nHowever, several European countries were involved in the situation in Cyprus at the time. The United Kingdom, Greece, and Turkey were all signatories to the Treaties of Guarantee and Alliance that were intended to ensure the independence, territorial integ

generated answers: ['No, the Greek Cypriots have not always constituted a majority in Cyprus. The demographics of Cyprus have changed over time due to various historical events, migrations, and conflicts. \n\nFor example, during the Ottoman period, there was a significant Turkish Cypriot population in Cyprus. However, after Cyprus gained independence from British colonial rule in 1960, tensions between the Greek Cypriot and Turkish Cypriot communities led to intercommunal violence and eventually resulted in the division of the island', "No, the Greek Cypriots have not always constituted a majority in Cyprus. Before independence from British rule in 1960, the island's population was made up of a mix of Greek Cypriots, Turkish Cypriots, Armenians, Maronites, and other communities. \n\nAfter independence, tensions between the Greek and Turkish communities escalated, leading to violence and eventually the division of the island in 1974. The Turkish invasion resulted in the de facto partiti

generated answers: ['During the Bronze Age in Cyprus, which lasted from around 2500 BC to 1050 BC, the island experienced significant cultural, technological, and societal developments. Cyprus was an important center of trade and commerce, as evidenced by the abundance of imported goods found in archaeological sites. \n\nThe Bronze Age in Cyprus is characterized by the emergence of advanced metallurgy, with copper being one of the most important resources on the island. Cyprus became a major producer and exporter of copper, which was highly sought after', 'The Bronze Age in Cyprus lasted from around 2500 BC to 1050 BC. During this time period, Cyprus was a center of trade and cultural exchange in the eastern Mediterranean region. The island was known for its production of copper and bronze, which were highly valued commodities in the ancient world. Cyprus was also home to advanced civilizations, such as the Mycenaeans and the Phoenicians, who left behind impressive archaeological sites

generated answers: ['Several factors contributed to the Turkish invasion of Cyprus in 1974:\n\n1. Ethnically motivated violence: Tensions between Greek Cypriots and Turkish Cypriots had been escalating since independence from British rule in 1960. The intercommunal violence between the two communities was a key factor that led to the Turkish invasion.\n\n2. The Greek military junta: In 1974, a military junta in Greece supported a coup by Greek nationalists in Cyprus, aiming to unite', 'There were several factors that contributed to the Turkish invasion of Cyprus in 1974:\n\n1. Ethnical tensions: Cyprus had a significant population of both Greek Cypriots and Turkish Cypriots, and tensions between the two communities had been escalating since the country gained independence in 1960. The Greek Cypriot majority sought to unify the island with Greece (Enosis), while the Turkish Cypriot minority favored partition or integration with Turkey (Taksim).\n\n2. Greek', '1. The growth of nationalis

...

generated answers: ['Cyprus played a significant role in the trade networks of the ancient Mediterranean as it was strategically located at the crossroads of Europe, Asia, and Africa. The island was renowned for its production of copper, which was highly sought after by civilizations throughout the region. Cyprus also produced other valuable commodities such as timber, wine, and olive oil.\n\nThe island served as a major hub for trade routes connecting the eastern Mediterranean with regions such as Egypt, Greece, Anatolia, and the Levant. Traders', 'Cyprus played a significant role in the trade networks of the ancient Mediterranean. As an island located at the crossroads of Europe, Asia, and Africa, Cyprus served as a crucial hub for maritime trade routes. The island was well-known for its abundant natural resources, including copper, timber, and agricultural products, which made it an attractive trading partner for many civilizations.\n\nCyprus also had strategic importance due to its

generated answers: ['In antiquity, Cyprus was influenced by several major powers, including:\n\n1. Ancient Greece: Greek city-states exerted influence over Cyprus, with colonial settlements established on the island, particularly during the Archaic and Classical periods.\n\n2. Persian Empire: Cyprus came under the control of the Persian Empire in the 6th century BCE, following their conquest of the island.\n\n3. Roman Empire: Cyprus was annexed by the Roman Republic in 58 BCE, becoming a Roman province shortly after.', 'The major powers that exerted influence over Cyprus in antiquity were the Egyptians, the Assyrians, the Persians, the Greeks, and the Romans.', 'In antiquity, Cyprus was influenced by several major powers, including:\n\n1. Ancient Greece: Cyprus was initially settled by Greek colonists in the 8th century BC and became an important part of the Greek world. Several city-states established colonies on the island, including Salamis, Kourion, and Marion.\n\n2. Persia: In the

generated answers: ["Cyprus came under Egyptian rule during the New Kingdom period through military conquest and expansion. In the late 15th century BCE, Egyptian pharaoh Thutmose III launched a series of campaigns to expand the Egyptian empire and assert control over various territories, including Cyprus. Thutmose III's forces successfully defeated the local Cypriot rulers and established Egyptian control over the island, which became a valuable strategic and economic asset for the Egyptian empire. Cyprus remained under Egyptian rule for several centuries, serving as an", 'Cyprus came under Egyptian rule during the New Kingdom period due to military campaigns launched by the pharaohs of Egypt. The most well-known campaign was led by Pharaoh Thutmose III in the 15th century BC, who conquered Cyprus and established it as a colony of Egypt. The Egyptian presence on the island was further solidified during the reign of other pharaohs, such as Amenhotep II and Ramesses III. trade and diplo

generated answers: ['There is a significant amount of evidence suggesting Mycenaean influence on Cyprus during the Late Bronze Age. Some of the key pieces of evidence include:\n\n1. Mycenaean pottery: Mycenaean pottery has been found at various archaeological sites on Cyprus, indicating trade and cultural exchange between the two regions. This pottery typically features characteristic Mycenaean decorations and motifs, such as spiral designs and marine motifs.\n\n2. Mycenaean-style tombs: Archaeologists have uncovered a number', 'There is significant evidence of Mycenaean influence on Cyprus, particularly in the Late Bronze Age (1600-1100 BCE). Some of the key pieces of evidence include:\n\n1. Mycenaean-style pottery: Archaeological excavations have uncovered numerous examples of Mycenaean-style pottery in Cyprus, such as stirrup jars, kraters, and kylixes. These ceramics feature distinctive Mycenaean motifs and designs, indicating trade and cultural exchange between Mycenaean Greece', 

generated answers: ['During the Greek Dark Ages (circa 1100-800 BCE), Cyprus played a significant role in the exchange of goods, ideas, and cultures throughout the eastern Mediterranean. The island was an important trading hub due to its strategic location and abundance of natural resources such as copper.\n\nCyprus was inhabited by various indigenous peoples, including the Eteocypriots and the Greek-speaking Achaean settlers who arrived in the late Bronze Age. During the Greek Dark Ages, Cyprus continued to maintain strong', 'During the Greek Dark Ages, Cyprus maintained close connections with the Greek world despite its geographical distance. Cyprus continued to trade with neighboring regions and is believed to have been a significant source of copper at this time, which was a valuable commodity for the Greeks.\n\nCyprus also continued to be influenced by Greek culture, as demonstrated by the presence of Greek pottery and other artifacts found on the island. The Cypriot city-kingdoms

In [13]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_EVL_3_new.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

generated answers: ['In the international arena, Cyprus is a member of several important organizations, including the United Nations, the European Union, and the Commonwealth of Nations. The country also participates in various international forums and conferences, where it discusses issues related to peace, security, and economic development.\n\nCyprus plays an active role in promoting peace and stability in the region, particularly in relation to the ongoing conflict between the Greek Cypriots and Turkish Cypriots on the island. The United Nations has been involved in', 'Cyprus is a country situated in the Eastern Mediterranean region, with a complex political and diplomatic landscape at both the international and national levels.\n\nInternationally, Cyprus has been a member of the United Nations since its independence in 1960. It is also a member of the European Union, the Council of Europe, and the Commonwealth of Nations. Cyprus has historically had a strong relationship with Gree

generated answers: ['The European Union did not exist in 1964 as it was only established in 1993. However, the member states of the European Economic Community (EEC), which later became part of the EU, expressed concerns and condemnation of the situation in Cyprus. They called for a peaceful resolution to the conflict and supported the efforts of the United Nations to mediate between the Greek Cypriot and Turkish Cypriot communities. Some EEC member states also provided humanitarian aid to the affected populations in Cyprus', 'When Cyprus was divided along ethnic lines in 1964 due to the EOKA terror campaign, the European Union did not exist in its current form. However, individual European countries and international organizations expressed concern and called for peace efforts to resolve the situation.\n\nThe United Nations and various European countries, including the United Kingdom, France, and Italy, played a role in trying to mediate and facilitate negotiations between the Greek a

generated answers: ['No, the Greek Cypriots have not always constituted a majority in Cyprus. Historically, the island of Cyprus has had a diverse population consisting of Greek Cypriots, Turkish Cypriots, Armenians, Maronites, and other smaller ethnic groups. \n\nDuring different periods of history, the population composition of Cyprus has varied due to factors such as migration, conquests, and conflicts. The Greek Cypriots have been the majority ethnic group in Cyprus for much of', 'No, the Greek Cypriots have not always constituted a majority in Cyprus. Prior to independence from British colonial rule in 1960, the island was inhabited by a mixed population of Greek Cypriots, Turkish Cypriots, and other ethnic groups. The population of Cyprus has fluctuated over the years due to various factors, including mass migrations, conflicts, and political changes. Today, the Greek Cypriots are the majority population in Cyprus, while the Turkish Cyp', "No, the Greek Cypriots have not always c

generated answers: ['During the Bronze Age in Cyprus, which lasted from around 2500 BC to 1050 BC, the island experienced significant advancements in metallurgy, trade, and culture. Cyprus became a hub for copper production and trade, leading to increased wealth and influence in the eastern Mediterranean region.\n\nThe Bronze Age in Cyprus is divided into three main phases: Early Bronze Age (2500-1900 BC), Middle Bronze Age (1900-1600 BC), and Late Bronze Age (1600-105', 'During the Bronze Age in Cyprus, which lasted from around 2500 to 1100 BC, the island experienced significant cultural and technological developments. Cyprus became a major center for copper production and trade, leading to the emergence of advanced metallurgy techniques. The people of Cyprus also engaged in long-distance trade with neighboring regions, exchanging goods such as pottery, textiles, and metalwork.\n\nThe Bronze Age in Cyprus saw the rise of sophisticated urban centers, with notable sites such as Enkomi, 

generated answers: ["Several factors contributed to the Turkish invasion of Cyprus in 1974, including:\n\n1. Ethnic tensions: The island of Cyprus was home to a significant Greek Cypriot majority and a Turkish Cypriot minority. Tensions between the two communities had been long-standing, and escalated following a coup d'etat by Greek nationalists in 1974, aimed at unifying Cyprus with Greece.\n\n2. Enosis and Taksim: Enosis was the Greek nationalist movement seeking union with Greece, while", 'The main causes of the Turkish invasion in Cyprus in 1974 were:\n\n1. The desire of Greek Cypriots for enosis (union) with Greece, which had been a longstanding aspiration among the Greek population of Cyprus.\n\n2. The rise of tensions between the Greek and Turkish Cypriot communities on the island, exacerbated by discriminatory practices against the Turkish Cypriots by the Greek Cypriot government.\n\n3. The military coup in Cyprus in 1974, led by Greek', '1. Nationalism: The Turkish Cypriots f

...

generated answers: ["Cyprus played a significant role in the trade networks of the ancient Mediterranean due to its strategic location. Situated at the crossroads of Europe, Asia, and Africa, Cyprus served as a key trading hub connecting various regions and facilitating the exchange of goods, ideas, and cultural practices.\n\nThe island's rich natural resources, including copper, timber, and agricultural products, made it an important center for trade in the ancient world. Cyprus was known for its high-quality copper mines, which were in high demand", "Cyprus played a significant role in the trade networks of the ancient Mediterranean as it was strategically located at the crossroads of Europe, Asia, and Africa. The island was known for its abundance of copper and timber, which made it a valuable trading hub for goods such as metals, pottery, textiles, and luxury items.\n\nCyprus was a major source of copper throughout the ancient world, with mines that were actively exploited from as 

generated answers: ['Some of the major powers that exerted influence over Cyprus in antiquity were:\n\n1. Ancient Greece - Various city-states from Greece, such as Athens, Sparta, and Corinth, established colonies and settlements on Cyprus and exerted influence over the island during the Archaic and Classical periods.\n\n2. Persia - The Achaemenid Persian Empire conquered Cyprus in 545 BC and held control over the island until its defeat by Alexander the Great in 333 BC.\n\n3. Egypt - The', 'Some of the major powers that exerted influence over Cyprus in antiquity include:\n\n1. The ancient Greeks, who established settlements and trade networks on the island and eventually ruled over it during the Archaic, Classical, and Hellenistic periods.\n\n2. The Phoenicians, who also established settlements and trade routes on the island and maintained a significant presence in Cyprus during the Iron Age.\n\n3. The Roman Empire, which annexed Cyprus in 58 BC and ruled over the island as a', 'The m

generated answers: ["Cyprus came under Egyptian rule during the New Kingdom period primarily through military conquest. The Egyptian pharaoh Thutmose III launched a series of military campaigns in the eastern Mediterranean region during the 15th century BCE, which included the invasion and conquest of Cyprus.\n\nThutmose III's campaigns were aimed at expanding Egypt's influence and control over the region, as well as securing important trade routes and resources. The Egyptian army successfully defeated the local Cypriot kingdoms and established Egyptian control over the island.\n\n", 'Cyprus came under Egyptian rule during the New Kingdom period through military conquest by the pharaohs of Egypt. The exact details of the conquest are not well-documented, but it is believed that the Egyptian forces invaded and conquered Cyprus sometime during the late 16th or early 15th century BC. The Egyptians established control over the island and incorporated it into their empire, establishing trad

generated answers: ['1. Archaeological finds: Excavations at sites such as Hala Sultan Tekke and Enkomi have revealed Mycenaean-style pottery, architecture, and other artifacts, indicating cultural and trade connections between Mycenaean Greece and Cyprus.\n\n2. Linear B tablets: Some Linear B tablets, a script used by the Mycenaeans, have been found in Cyprus, suggesting communication and possible administrative influence by the Mycenaeans on the island.\n\n3. Mycenaean', '1. Mycenaean pottery has been found in various archaeological sites across Cyprus, indicating trade and cultural exchange between Mycenaean Greece and Cyprus.\n\n2. Mycenaean-style architecture, such as Cyclopean walls and megaron-style buildings, have been identified in some settlements on Cyprus.\n\n3. Mycenaean inscriptions and Linear B tablets have been discovered on the island, hinting at Mycenaean administrative presence in Cyprus.\n\n4. The presence of Myc', 'There is evidence of Mycenaean influence on Cyprus

generated answers: ['During the Greek Dark Ages (1100-800 BC), Cyprus maintained its own distinct cultural identity while also being influenced by the surrounding Greek world. The island was an important center for trade and commerce, particularly in the eastern Mediterranean, and it continued to be a hub for cultural exchange between the Greek mainland, the Near East, and Egypt.\n\nCyprus participated in the Greek Dark Ages through its trade networks and connections with other cultures. The island was an important source of copper, which was in high demand', 'During the Greek Dark Ages (1100-800 BC), Cyprus maintained close cultural and trade connections with the Greek world despite the decline of the Mycenaean civilization on the mainland. The island continued to be an important hub for trade and cultural exchange, although its own society also experienced some decline.\n\nCyprus participated in the Greek Dark Ages through its trade networks, which continued to connect the island wit

Here we run again the code to check for new uncertainty values in case 3: 

### 4rt case:
- Model: gpt-4
- Estimaotr: EigValLaplacian

In [6]:
model = BlackboxModel(
    api_key,
    'gpt-4'
)

estimator = EigValLaplacian(verbose=True)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We store the data from this case in a csv file named: "results_EVL_4.csv"

In [11]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_EVL_4.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

generated answers: ['Cyprus, an island country in the Eastern Mediterranean, is known for both its national and international arenas related to sport, political, and cultural events. \n\n1. GSP Stadium: This is the largest football stadium in Cyprus, with a capacity for 22,859 people. It serves as the home ground for APOEL FC and the Cyprus National Football Team.\n\n2. Antonis Papadopoulos Stadium: It is the top league football stadium located in Larnaca and is mostly used for', 'Cyprus is a small island nation located in the Eastern Mediterranean. Despite its size, it has significant influences in both national and international arenas due to its strategic location and complex political history.\n\nInternational Arena: \n\nCyprus has been a member of the United Nations since its independence from British colonial rule in 1960. It is also a member of the Commonwealth, the World Bank, and the International Monetary Fund. In recent years, it has been most active in international politic

generated answers: ['The European Union, or the precursor to it, did not take significant action during the 1964 division of Cyprus because the EU as it is known today did not exist. Its previous incarnation, the European Economic Community (EEC), had only six members and largely focused on economic cooperation rather than political or security issues, which were left to organizations such as NATO and the United Nations. Apart from that, Cyprus had not yet become a member of the EEC or the EU, it only joined in', 'The events in Cyprus during the mid-1960s took place over a decade before the country would join the European Union (EU). At that time, the EU was still known as the European Economic Community (EEC) and only consisted of six countries (Belgium, Germany, France, Italy, Luxembourg and the Netherlands). Therefore, the EEC did not have a direct response or involvement in these events within Cyprus.\n\nHowever, the international community, including countries that would later for

generated answers: ["Yes, Greek Cypriots have historically been the majority ethnic group in Cyprus for centuries. Since the island became an independent republic in 1960, Greek Cypriots have consistently constituted between 70-80% of the island's population, with Turkish Cypriots being the second largest ethnic group. Prior to 1974, the communities were interspersed across the whole island. Today, the majority of Greek Cypriots live in the south of the island, while", 'Yes, Greek Cypriots have always constituted the majority population in Cyprus. However, the island has also been home to significant Turkish Cypriot community, who traditionally have been the minority. Other minor ethnic groups include Armenians, Maronites and others.', "Yes, Greek Cypriots have historically constituted a majority in Cyprus. When Cyprus became an independent country in 1960, Greek Cypriots accounted for around 80% of the island's population, with Turkish Cypriots making up most of the remaining 20%. Tod

generated answers: ["The Bronze Age in Cyprus approximately ranges from 2400 to 1050 BC. This time period is often broken into the Early, Middle, and Late Cypriot periods.\n\n1. Early Cypriot Period (2300 – 1900 BC) : The pottery during this period was highly decorated. Copper was first exploited during this period leading to Cyprus's later renown as a major source of copper. \n\n2. Middle Cypriot Period (1900 – 1600 BC): This", 'The Bronze Age in Cyprus, also known as the Cypriot Bronze Age, stretched from approximately 2500 BC to 1050 BC. It is marked by the production and use of bronze tools, weapons, and decorative items. It is typically divided into the Early Cypriot (EC), Middle Cypriot (MC), and Late Cypriot (LC) periods.\n\nThis era witnessed significant cultural progress and development on the island. Cyprus had close cultural ties with the Near East, especially', "The Bronze Age in Cyprus is a period of the island's history that stretches from approximately 2500 to 1050 BC. I

generated answers: ['The causes for the 1974 Turkish invasion in Cyprus can be attributed to a number of historical, political, and ethnic tension factors:\n\n1. Ethnic Conflict: The island of Cyprus had a majority of Greek Cypriots and a substantial minority of Turkish Cypriots. There had been ethnic tensions and violence between these two communities since the 1950s. \n\n2. Independence and Union with Greece: Cyprus gained independence from Britain in 1960, but the constitution was designed to balance', "The Turkish invasion of Cyprus, also known as the Cyprus Peace Operation in Turkey and the Attila Operation, occurred in 1974 and was the result of various political and ethnic tensions. Here are some primary causes:\n\n1. Ethnopolitical Conflicts: Cyprus was primarily inhabited by Greek and Turkish Cypriots. Following the country's independence from British rule in 1960, disputes arose between these two communities over their constitutional rights, sparking intermittent periods of v

...

generated answers: ['Cyprus played a crucial role in the trade networks of the ancient Mediterranean due to its strategic location and resources. It was a primary source of copper, which attracted many traders and civilizations. \n\nDuring the Bronze Age, Cypriot metalwork was in high demand and was traded widely across the Mediterranean. Cyprus was at the crossroads of major trading routes between Europe, Asia, and Africa, thus facilitating cultural exchange and trade of commodities. \n\nFurthermore, Cyprus was renowned for its production of fine pottery and', 'Cyprus played a major role in the trade networks of the ancient Mediterranean due to its strategic location and abundant resources.\n\n1. Strategic Location: Positioned at the crossroads of Europe, Asia, and Africa, Cyprus served as a crucial transit point for traders and explorers traveling among these continents. This gave Cyprus a significant role in the trade networks of the Mediterranean. \n\n2. Copper and Timber Resources

generated answers: ['The major powers that exerted influence over Cyprus in antiquity were:\n\n1. Mycenaeans/Achaeans (Greeks) - Around 1200 BC, the Mycenaean Greeks settled in Cyprus. They introduced the Greek language and culture to the island, influences of which can still be seen in Cyprus today.\n\n2. Assyrians - In the 8th century BC, Assyrians conquered Cyprus and ruled it for around a century. \n\n3. Egyptians - For about', '1. Egyptian Empire: The New Kingdom of Egypt controlled Cyprus during the Late Bronze Age. The island produced copper for the Egyptian empire during this period.\n\n2. Mycenaean Greeks: The Mycenaean used the island as a commercial and military outpost around 1400-1100 BC. This marked the beginning of Greek influence on Cyprus. \n\n3. Hittite Empire: The Hittite held some control over Cyprus during the Late Bronze Age. \n\n4. Assyrian Empire', 'Cyprus was a strategically important island in antiquity, often coveted by various powerful civilizations. The maj

generated answers: ['The New Kingdom period, also referred to as the Egyptian Empire, occurred between the 16th century BC to the 11th century BC. During this period, Egypt became a force to be reckoned with in Ancient Near East. Massive conquests and diplomacy expanded the boundaries of the Egyptian empire. \n\nCyprus, an island located in the eastern Mediterranean, had close ties with Egypt as early as the Old Kingdom period. The island provided precious resources such as copper and timber, necessary for Egyptian construction', "In the Late Bronze Age (1550–1200 BC), Cyprus had vital commercial ties with Egypt. The island produced and exported copper, one of the main materials used in the ancient world, and shared close political and cultural contacts with the Egyptian civilization. During this period, Egypt was at its peak, controlling a vast empire under the New Kingdom (1550–1069 BC).\n\nHowever, it's important to note that Cyprus was not directly ruled by Egypt. Rather, the relat

generated answers: ['The Mycenaean civilization, which thrived in the late Bronze Age in Greece (approx. 1600–1100 BCE), has evidence of influence on Cyprus. Here are a handful of evidences:\n\n1. Pottery: Extensive quantities of Mycenaean pottery have been discovered in Cyprus, which were most likely brought by Mycenaean merchants or produced locally by Mycenaean immigrants. The distinct style of the pottery shows a direct influence on Cypriot ceramics.\n\n', 'The Mycenaean civilization was a prominent Bronze Age civilization in Greece that had significant influence across the Mediterranean, including on the Island of Cyprus. Here are a few key pieces of evidence demonstrating their influence:\n\n1. Mycenaean Pottery: One of the most significant pieces of evidence is the widespread discovery of Mycenaean pottery on Cyprus, dating from around 1400-1200 BC. This suggests that there were extensive trading links between the two civilizations. \n\n2. Language:', 'Several forms of evidence 

generated answers: ["The Greek Dark Ages, also known as the Homeric Age or the Geometric period, occurred from about 1100 B.C. to 800 B.C. This period is marked by the disappearance of linear writing systems, the destruction of many settlements, and general social and economic regression.\n\nWhile the Greek mainland went through the Dark Ages, Cyprus did not experience the same kind of regression. Rather, Cyprus maintained relationships with the cultures in the Near East and Egypt, and there's evidence of a flourishing", 'The Greek Dark Ages, also known as the Geometric or Homeric Age, was a period of time in ancient Greece from the end of the Mycenaean palatial civilization around 1100 BC to the first signs of the Greek city-states in the 9th century BC.\n\nDuring this period, Cyprus, known as Alashiya in ancient times, was one of the few parts of the Greek world that maintained a high level of civilization. It appears to have been less affected by the', 'The Greek Dark Ages is a peri

Check values for Laplacian for both model: is it again the gpt 4 works better than gpt 3.5

It looks like that while using Laplacian as the estimator the gpt-3.5 works better than gpt-4!!! Crazy 

Here we run again the code to check for new uncertainty values in case 4: 

In [7]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_EVL_4_new.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

generated answers: ['Cyprus, a nation located in the far eastern fringes of the Mediterranean Sea, offers both national and international arenas that are used for various events, ranging from sports, musical concerts, conferences and other important public gatherings. \n\n1. GSP Stadium: This is the largest football stadium in Cyprus, located in Nicosia, with a capacity of around 22,859 people. The stadium is home to APOEL, Olympiakos Nicosia and the Cyprus national football team.\n\n', "Cyprus is an island country located in the Eastern Mediterranean. It is divided politically into two entities: the Republic of Cyprus, a member of the United Nations and the European Union, and the Turkish Republic of Northern Cyprus, which is only recognized by Turkey.\n\nInternational Arena:\n\n1. Cyprus's location in the Mediterranean and its geopolitical situation puts it at the center of many global discussions, particularly those related to Europe, the Middle East, and the UN.\n2. As a member of 

generated answers: ["The European Union (EU) as we know it today wasn't founded until 1993, so it didn't react as a unified entity to the division of Cyprus in 1964. Prior to 1993, the precursor to the EU was the European Economic Community (EEC). While it dealt with economic issues, politics and violence, such as that in Cyprus, weren't in its scope.\n\nNonetheless, a more general European reaction to the division of Cyprus in 1964 can be", 'The European Union (EU) did not exist in 1964, it was established on November 7, 1993. Therefore, it could not have had any reaction to the events in Cyprus in 1964. At the time, the closest equivalent to the EU would have been the European Economic Community (EEC). They did not have much of a direct reaction, as their focus was primarily on economic cooperation rather than political issues. \n\nThe international community as a whole saw the situation with', "The European Union (EU) was not yet fully formed during the 1964 division of Cyprus. The 

generated answers: ["Yes, Greek Cypriots have generally always constituted a majority in Cyprus. Historically, the island has been closely linked to Greece and has a predominantly Greek culture. However, there has also been a significant Turkish Cypriot community, particularly since the Ottoman conquest in the late 16th century. Tensions between the Greek and Turkish communities came to a head in the late 20th century, leading to the island's current division, but Greek Cypriots still constitute a majority in", "Yes, Greek Cypriots have primarily constituted a majority of the population in Cyprus. However, it should be noted that the island has a significant minority of Turkish Cypriots, and it has a history of tension between the two communities. It's also important to acknowledge how different periods of rule (Ottoman, British, and others) have influenced the demographic make-up of Cyprus.", "Yes, Greek Cypriots have always constituted a majority in the island of Cyprus, including bo

generated answers: ['The Bronze Age in Cyprus, also known as the Chalcolithic period, is said to have started around 2500 BC and ended around 1050 BC. During this time, the island was known as Alashiya. The Bronze Age in Cyprus was characterized by the production of bronze tools, weapons, and ornaments. The local inhabitants of the island developed sophisticated methods of mining and metalworking which allowed them to trade extensively with other civilizations around the Mediterranean.\n\nIt was a period marked by', 'The Bronze Age in Cyprus is also referred to as the Cypriot Bronze Age. It started around 2500 BCE with the introduction of bronze for tool making and ended around 1050 BCE. This period in Cyprus was characterized by the production and use of bronze items, increased trade activities especially with surrounding regions like the Aegean, and advancements in pottery and other crafts.\n\nThe Bronze Age is divided into different periods, including the Early Cypriot period (2500-

generated answers: ['Along with a complex historical and political context, here are the prime causes of the Turkish invasion in Cyprus:\n\n1. Ethnic Tensions: The primary cause of the Turkish invasion was ethnic tension between Greek and Turkish Cypriots. From the time Cyprus gained independence from Britain in 1960, conflicts between Greek Cypriots (who aimed to unify Cyprus with Greece, also known as "Enosis") and Turkish Cypriots (who wanted the partition of the island, in a concept', 'The Turkish invasion in Cyprus in 1974, also known as the Cyprus Peace Operation in Turkey, was a result of various legal, political, and historical conflicts. Cyprus has historically been a nation of mixed ethnicities, dominated by Greeks and Turks. When Cyprus gained its independence from Britain in 1960, it faced many crises due to its constitutional framework favoring bicommunal democracy. Three main causes are typically attributed to the Turkish invasion:\n\n1. Ethnic Tension: The main reason be

...

generated answers: ['Cyprus played a crucial role in the trade networks of the ancient Mediterranean. It was a hub of trade and commerce due to its strategic geographical location between three continents: Europe, Asia, and Africa. The island was rich in resources, especially copper, which was heavily sought after in the Bronze Age. The name Cyprus is even believed to originate from the ancient word for copper. \n\nThe Cypriots also mastered the craft of shipbuilding, allowing them to control maritime trade routes. This combination of', "Cyprus played a significant role in the trade networks of the ancient Mediterranean. It was strategically located at the crossroads between Europe, Asia, and Africa, which made it an important center for trade and exchange.\n\nMaterials such as copper, timber, and textiles were exported from Cyprus to various regions around the Mediterranean. Copper, in particular, was one of the island's most important exports and it is even named after the island (fr

generated answers: ['In antiquity, the major powers that exerted influence over Cyprus were:\n\n1. Mycenaean Greeks: Around 1400 BC, Mycenaean Greeks settled in Cyprus and introduced the Greek language and culture.\n\n2. Assyrians: The Assyrians exerted control over Cyprus around 700 BC.\n\n3. Egyptians: During various periods, particularly around 570 BC during the reign of Amasis, Cyprus was under Egyptian control.\n\n4. Persians: The Achaemenid Empire', 'In antiquity, Cyprus was influenced by numerous major powers, including:\n\n1. Mycenaean Greeks: The first wave of Greeks arrived in Cyprus around 1400 BC, introducing the Greek language and culture.\n\n2. Phoenicians: They established numerous colonies and enclaves on Cyprus during the first millennium BC, influencing language, art, and commerce.\n\n3. Egypt: Cyprus was part of the Egyptian Empire under the Ptolemaic dynasty from 323 to 30 BC.\n\n', "In antiquity, the major powers that exerted influence over Cyprus were:\n\n1. Mycen

generated answers: ["During the New Kingdom period (1550–1077 BC), Egypt was a prominent regional power with significant influence extending beyond its borders. Egypt didn't really establish direct rule over Cyprus, but rather had a strong influence over the island due to their extensive trade relationships and diplomatic ties. \n\nCyprus was an important source of copper, a material essential for weaponry and tools, which the Egyptians had a great need for. Evidence from archaeological discoveries like the Amarna letters, which are a collection of international correspondence", 'The Egyptian influence over Cyprus during the New Kingdom Period (1550–1077 BC) did not come as a result of direct rule or annexation. Historical texts and archeological evidence suggest that Cyprus had strong trading and diplomatic ties with Egypt during this period. The island served as a crucial trading hub for copper, a resource that was vital for the Egyptians.\n\nCypriot pottery, jewelry, and other artif

generated answers: ['There is plenty of archaeological evidence of Mycenaean influence on Cyprus, dating from the Late Bronze Age (approximately 1600-1100 BC). \n\n1. Pottery: The most prominent evidence of Mycenaean influence on Cyprus lies in the widespread presence of Mycenaean pottery throughout the island. The pottery was generally imported from mainland Greece, which indicates robust trade relations. Later on, local Cypriot potters began imitating Mycenaean styles, demonstrating a', 'The Mycenaean civilization was a major influence on the Mediterranean and significantly impacted various regions including Cyprus. The evidence of Mycenaean influence on Cyprus is mainly seen in various archaeological findings.\n\n1. Pottery: Mycenaean pottery, dating back to the Late Bronze Age, has been found extensively in various parts of Cyprus. The discovery of such pottery insinuates a significant Mycenaean presence or strong trade relations.\n\n2. Architecture: There are similarities between 

generated answers: ['Cyprus was not just a passive spectator during the Greek Dark Ages, a period of decline lasting from 1100 B.C. to 800 B.C following the fall of the Mycenaean civilization. This time is often characterized by depopulation, migration, and limited literacy.\n\nCyprus, being an important trading center in the Mediterranean, had links with various civilizations and cultures during this period and played a dynamic role. While the Aegean region (Greek mainland and islands) was immersed', 'The Greek Dark Ages, also known as the Homeric Age, refers to the period of Greek history from the end of the Mycenaean palatial civilization around 1100 BC to the first signs of the Greek city-states in the 9th century BC.\n\nDuring this period, Cyprus was largely disconnected from the major centers of Greek civilization in mainland Greece. However, there is archaeological evidence to suggest that Cyprus maintained some contact with the Aegean world. \n\nCyprus was not entirely engulf',

## GPT-4o

As the last part of this notebook we want to check all the previous methods with a new model that has just been released on the 14th of may of 2024 named GPT-4o:

First with Lexical Similarity

In [11]:
model = BlackboxModel(
    api_key,
    'gpt-4o-2024-05-13'
)

estimator = LexicalSimilarity()

In [12]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_LS_4o.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

It takes 12 minutes to run the faster of the 3 for lexical similarity

And then the same for EigValLaplacian

In [9]:
model = BlackboxModel(
    api_key,
    'gpt-4o-2024-05-13'
)

estimator = EigValLaplacian(verbose=True)

In [10]:
# Estimate uncertainty for each prompt and save the results
output_filename = 'results_EVL_4o.csv'
estimate_uncertainty_for_prompts(model, estimator, input_messages, output_filename)

generated answers: ['Cyprus, an island nation in the Eastern Mediterranean, plays diverse roles in both international and national arenas. Here’s an overview of its positions and dynamics in these arenas:\n\n### International Arena:\n1. **European Union**:\n   - **Membership**: Cyprus joined the European Union (EU) in 2004. As an EU member state, Cyprus participates in various EU institutions and policy-making processes.\n   - **Eurozone**: It adopted the euro as its currency in 2008,', 'Cyprus is an island nation in the Eastern Mediterranean, and its international and national arenas are shaped by its geopolitical position, historical tensions, and membership in various international organizations. Here’s an overview of both the international and national aspects of Cyprus:\n\n### International Arena\n\n#### **European Union (EU)**\n1. **Membership**: Cyprus joined the European Union in 2004, which has significantly shaped its economic and political landscape.\n2. **Eurozone**: Cyprus

generated answers: ["The European Union (EU) as we know it today did not exist in 1964. The predecessor of the EU was the European Economic Community (EEC), which was primarily focused on economic cooperation and integration among its member states. Cyprus only became a member of the EU in 2004, so Cyprus and its internal issues were not within the direct scope of the EEC's mandate in 1964. \n\nThe division of Cyprus along ethnic lines in 1964 primarily involved conflicts between the", 'The division of Cyprus along ethnic lines in 1964 was a complex issue involving multiple parties and a backdrop of inter-communal violence between Greek Cypriots and Turkish Cypriots. This situation, often attributed to the actions of EOKA (a Greek Cypriot nationalist guerrilla organization), led to severe tensions and conflict on the island.\n\nHowever, it is important to note that the European Union (EU) as we know it did not exist in 1964. At that time', 'The European Union (EU) as we know it today d

generated answers: ['Yes, Greek Cypriots have traditionally constituted the majority population in Cyprus. Historically, the demographic composition of the island has been primarily divided between Greek Cypriots, who are predominantly Greek Orthodox Christians, and Turkish Cypriots, who are predominantly Muslims.\n\nGreek Cypriots have constituted around 70-80% of the population, while Turkish Cypriots have made up approximately 18-20%. The exact proportions have varied over time due to factors such as migration, birth rates, and political', 'No, Greek Cypriots have not always constituted a majority in Cyprus uniformly throughout its entire history, but they have been the majority ethnic group for several centuries. \n\nThe demographic composition of Cyprus has varied over different historical periods due to invasions, migrations, and political changes. Since the Byzantine era (approximately 4th to 12th centuries CE), Greek Cypriots have been the predominant ethnic group on the island

generated answers: ["The Bronze Age in Cyprus is a significant period marked by substantial developments in the island's social, economic, and cultural life. This period extends approximately from 2500 BC to 1050 BC. Here's a more detailed look at the key phases and characteristics of the Bronze Age in Cyprus:\n\n### Early Bronze Age (c. 2500–1900 BC)\n- **Culture and Society**: The Early Bronze Age saw the establishment of small village communities. The island's inhabitants were engaged in agriculture", "The Bronze Age in Cyprus, spanning approximately from 2500 to 1050 BCE, marks a period of significant development and transition. This era saw the island emerge as an important center for trade and cultural exchange in the Eastern Mediterranean. \n\n### Early Bronze Age (2500-1900 BCE)\nDuring the Early Bronze Age, Cyprus was relatively isolated with small, independent settlements. The island's inhabitants were primarily engaged in agriculture, animal husbandry, and basic crafts.\n\n#

generated answers: ["The Turkish invasion of Cyprus in 1974 was a complex event with multiple interrelated causes rooted in the island's historical, ethnic, and political context. Here are the main causes:\n\n1. **Ethnic Tensions and Intercommunal Violence**: Cyprus has a mixed population of Greek Cypriots and Turkish Cypriots. Historical tensions between these communities had been escalating since the 1950s, fueled by nationalist movements like the Greek Cypriot desire for Enosis (union with Greece", 'The Turkish invasion of Cyprus, which occurred in 1974, was primarily driven by a combination of complex political, ethnic, and international factors. Here are some key causes:\n\n1. **Ethnic Tensions and Intercommunal Violence**: Cyprus has a long history of ethnic tension between the Greek Cypriots and Turkish Cypriots. In the years leading up to the invasion, there were numerous incidents of violence and discrimination between the two communities, exacerbating mutual distrust and host

...

generated answers: ['Cyprus played a significant and multifaceted role in the trade networks of the ancient Mediterranean due to its strategic location, natural resources, and cultural interactions.\n\n1. **Strategic Location**: Situated at the crossroads of major maritime routes connecting the Aegean Sea, the Levant, and Egypt, Cyprus served as a crucial hub for trade and cultural exchange between various civilizations, including the Minoans, Mycenaeans, Egyptians, Hittites, Phoenicians, Greeks, and', "Cyprus played a significant role in the trade networks of the ancient Mediterranean due to its strategic location, abundant natural resources, and advanced craftsmanship. Here are some key aspects of Cyprus's role in these networks:\n\n1. **Strategic Location**: Cyprus is situated at the crossroads of major maritime routes connecting the Aegean, the Near East, and Egypt. This made it a crucial hub for trade and cultural exchange among these regions.\n\n2. **Copper Production**: One of t

generated answers: ['Throughout antiquity, Cyprus was influenced and controlled by several major powers due to its strategic location in the eastern Mediterranean. The key powers that exerted influence over Cyprus during this period include:\n\n1. **Mycenaeans (ca. 1400-1100 BCE)**: Greek settlers, likely from the Mycenaean civilization, began to settle in Cyprus, introducing Greek language, culture, and religion.\n\n2. **Assyria (ca. 8th century BCE)**: During', 'Throughout antiquity, Cyprus was influenced and controlled by several major powers due to its strategic location in the eastern Mediterranean. These included:\n\n1. **Mycenaean Greeks**: The influence of Mycenaean Greeks in Cyprus dates back to around the 12th century BCE, when Greek settlers began to establish themselves on the island.\n\n2. **Assyrians**: In the 8th century BCE, Cyprus came under the influence of the Assyrian Empire, which extended its control', 'Throughout antiquity, the island of Cyprus was influenced and

generated answers: ["Cyprus came under Egyptian influence, though not direct rule, during the New Kingdom period (circa 1550–1070 BCE), particularly during the height of Egypt's imperial power. This period saw Egypt exercising considerable economic and political influence over the Eastern Mediterranean region.\n\n1. **Thutmose III and the Empire's Expansion**: One of the most prominent pharaohs of the New Kingdom was Thutmose III, who reigned from 1479 to 1425 BCE. Under", "During the New Kingdom period, particularly under the reign of Pharaoh Thutmose III (circa 1479–1425 BCE), Egypt expanded its influence and dominance throughout the Eastern Mediterranean region. While there isn't direct evidence that Cyprus came under complete Egyptian rule, it is widely recognized that Egypt established a commercial and, to some extent, political influence over Cyprus during this time.\n\nThutmose III conducted numerous military campaigns into the Levant, reaching as far as the Euphrates River. The

generated answers: ['The Mycenaean influence on Cyprus, particularly during the Late Bronze Age (circa 1600–1100 BCE), is well-documented through a variety of archaeological and material evidence. Here are some key indicators:\n\n1. **Pottery**: One of the most prominent pieces of evidence is the presence of Mycenaean pottery styles on the island. This includes both imported Mycenaean ware and locally produced imitations that show Mycenaean stylistic features. The widespread', 'The Mycenaean civilization, which flourished during the Late Bronze Age (circa 1600–1100 BCE) in mainland Greece, had significant influence on Cyprus. Evidence of this influence encompasses various archaeological findings, including:\n\n### Archaeological Evidence:\n1. **Pottery**: It is one of the most telling indicators of Mycenaean presence and influence. Large quantities of Mycenaean-style pottery have been found at numerous sites across Cyprus, such as Enkomi,', 'Evidence of Mycenaean influence on Cyprus is

generated answers: ['The Greek Dark Ages, typically dated from around 1100 to 800 BCE, was a period of significant social, economic, and cultural regression and transformation in the ancient Greek world following the collapse of the Mycenaean civilization. Cyprus, an island in the eastern Mediterranean, experienced its own set of transformations during this period.\n\n1. **Cultural and Population Shifts**: During the Greek Dark Ages, Cyprus saw an influx of Greek-speaking peoples, including Achaeans and later Dori', 'The Greek Dark Ages, which stretched roughly from the 12th to the 8th century BCE, were marked by significant social, political, and economic changes following the collapse of the Mycenaean civilization. Cyprus, being geographically situated at a crucial crossroads in the Eastern Mediterranean, played a unique role during this period.\n\n1. **Continuity and Settlement**: Unlike many regions in mainland Greece, Cyprus did not experience a dramatic decline or complete abando